# Finding Nearest pair of Latitude and Longitude match

In [ ]:
!pip install geopandas

     |████████████████████████████████| 994 kB 2.7 MB/s 
     |████████████████████████████████| 6.3 MB 55.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 


In [ ]:
# imports 
import pandas as pd
from shapely.ops import nearest_points
import geopandas

In [ ]:
# upload data 
province_df = pd.read_csv('/content/Province_data.csv', delimiter= ";")
weather_df = pd.read_csv('/content/weather_knmi_stations_2018.csv', delimiter= ";")
yield_df = pd.read_csv('/content/Yield_data_cbs.csv', delimiter= ";")

In [ ]:
province_df.head(1)

,Geo Point,Geo Shape,Year,Provincie code,Provincie name,Gemeente code,Gemeente name,ISO 3166-3 Area code,Type,Gemeente code (with prefix)
0,"51.5131976789,4.42451793279","{""type"": ""Polygon"", ""coordinates"": [[[4.391083...",2021,30,Noord-Brabant,1674,Roosendaal,NLD,Gemeente,GM1674


In [ ]:
weather_df.head(1)

,STN,LON,LAT,ALT,NAME
0,209,4.518,52.465,0.0,IJMOND


In [ ]:
yield_df.head(1)

,ID,FarmTypes,Regions,Periods,NumberOfFarmsTotal_1,LandUseTotal_2,UtilisedAgriculturalAreaUAA_3,OtherLand_4,LandUseTotal_5,UtilisedAgriculturalAreaUAA_6,OtherLand_7,ArableLandTotal_8,PotatoesTotal_9,WarePotatoes_10,SeedPotatoes_11,StarchPotatoes_12,VegetablesArable_13,Cereals_14,GrassSeed_15,IndustrialCrops_16,Pulses_17,SugarBeets_18,OtherArableCrops_19,FallowLand_20,ArableLandTotal_21,PotatoesTotal_22,WarePotatoes_23,SeedPotatoes_24,StarchPotatoes_25,VegetablesArable_26,Cereals_27,GrassSeed_28,IndustrialCrops_29,Pulses_30,SugarBeets_31,OtherArableCrops_32,FallowLand_33,HorticultureInTheOpenTotal_34,FlowerBulbs_35,FloriculturalCrops_36,...,SucklingCows2Years_110,Bulls2YearsTotal_111,BullsForBreeding2Years_112,BullsForFattening2Years_113,Sheep_114,GoatsTotal_115,GoatsForMilking_116,OtherGoats_117,EquidaeTotal_118,Horses_119,Ponies_120,PigsTotal_121,Piglets_122,PigsForBreeding_123,PigsForFattening_124,ChickensTotal_125,LayingHens_126,ParentBirdsOfLayingHens_127,Broilers_128,ParentBirdsOfBroilers_129,Turkeys_130,DucksForFattening_131,OtherPoultry_132,Rabbits_133,FurredAnimals_134,HousedAnimalsTotal_135,PigsTotal_136,Piglets_137,PigsForBreeding_138,PigsForFattening_139,ChickensTotal_140,LayingHens_141,ParentBirdsOfLayingHens_142,Broilers_143,ParentBirdsOfBroilers_144,Turkeys_145,DucksForFattening_146,OtherPoultry_147,Rabbits_148,FurredAnimals_149
0,2877,A009481,NL01,2000JJ00,14799,52377452.0,48116261,4261191.0,14799.0,14799,10849.0,41586251,12343194,5476758,3252840,3613596,3065634,14173151,1620721,887169,183888,7468703,652213,1191578,12390,8095,5342,2197,1802,3677,10347,1839,1356,420,9501,1139,3153,199902,18098,17269,...,395.0,NaN,48.0,89.0,899,134,10,127,1066,693,570,21131,779,472,19880,139750,97600,NaN,42000,150,0,75,1278,1576,1847,159,75,5,11,66,70,68,NaN,2,1,0,1,9,3,3


# GET NEAREST WEATHER STATION FOR EACH PROVINCE 

In [ ]:
gpd = geopandas.GeoDataFrame(
    weather_df, geometry=geopandas.points_from_xy(weather_df['LON   '], weather_df.LAT))


In [ ]:
gpd.head(1)

,STN,LON,LAT,ALT,NAME,geometry,Nearest
0,209,4.518,52.465,0.0,IJMOND,POINT (4.51800 52.46500),Noord-Holland


In [ ]:

province_df[['LAT', 'LON']] = province_df['Geo Point'].str.split(',', expand=True)

In [ ]:
gpd2 = geopandas.GeoDataFrame(
    province_df, geometry=geopandas.points_from_xy(province_df.LON, province_df.LAT))

In [ ]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 
pts3 = gpd2.geometry.unary_union
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
     nearest = gpd2.geometry == nearest_points(point, pts)[1]
     return gpd2[nearest]['Provincie name'].to_numpy()[0]
gpd['Nearest'] = gpd.apply(lambda row: near(row.geometry), axis=1)


In [ ]:
gdf.Nearest.unique()

array(['Noord-Holland', 'Zuid-Holland', 'Fryslân', 'Flevoland', 'Utrecht',
       'Overijssel', 'Gelderland', 'Drenthe', 'Groningen', 'Zeeland',
       'Noord-Brabant', 'Limburg'], dtype=object)

In [ ]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 
pts3 = gpd.geometry.unary_union
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
     nearest = gpd.geometry == nearest_points(point, pts)[1]
     return gpd[nearest]['NAME'].to_numpy()[0]
gpd2['Nearest'] = gpd2.apply(lambda row: near(row.geometry), axis=1)


# MAP NEAREST WEATHER STATION TO PROVINCE IN YIELD DATA

In [ ]:
yield_df.head(1)

,ID,FarmTypes,Regions,Periods,NumberOfFarmsTotal_1,LandUseTotal_2,UtilisedAgriculturalAreaUAA_3,OtherLand_4,LandUseTotal_5,UtilisedAgriculturalAreaUAA_6,OtherLand_7,ArableLandTotal_8,PotatoesTotal_9,WarePotatoes_10,SeedPotatoes_11,StarchPotatoes_12,VegetablesArable_13,Cereals_14,GrassSeed_15,IndustrialCrops_16,Pulses_17,SugarBeets_18,OtherArableCrops_19,FallowLand_20,ArableLandTotal_21,PotatoesTotal_22,WarePotatoes_23,SeedPotatoes_24,StarchPotatoes_25,VegetablesArable_26,Cereals_27,GrassSeed_28,IndustrialCrops_29,Pulses_30,SugarBeets_31,OtherArableCrops_32,FallowLand_33,HorticultureInTheOpenTotal_34,FlowerBulbs_35,FloriculturalCrops_36,...,SucklingCows2Years_110,Bulls2YearsTotal_111,BullsForBreeding2Years_112,BullsForFattening2Years_113,Sheep_114,GoatsTotal_115,GoatsForMilking_116,OtherGoats_117,EquidaeTotal_118,Horses_119,Ponies_120,PigsTotal_121,Piglets_122,PigsForBreeding_123,PigsForFattening_124,ChickensTotal_125,LayingHens_126,ParentBirdsOfLayingHens_127,Broilers_128,ParentBirdsOfBroilers_129,Turkeys_130,DucksForFattening_131,OtherPoultry_132,Rabbits_133,FurredAnimals_134,HousedAnimalsTotal_135,PigsTotal_136,Piglets_137,PigsForBreeding_138,PigsForFattening_139,ChickensTotal_140,LayingHens_141,ParentBirdsOfLayingHens_142,Broilers_143,ParentBirdsOfBroilers_144,Turkeys_145,DucksForFattening_146,OtherPoultry_147,Rabbits_148,FurredAnimals_149
0,2877,A009481,NL01,2000JJ00,14799,52377452.0,48116261,4261191.0,14799.0,14799,10849.0,41586251,12343194,5476758,3252840,3613596,3065634,14173151,1620721,887169,183888,7468703,652213,1191578,12390,8095,5342,2197,1802,3677,10347,1839,1356,420,9501,1139,3153,199902,18098,17269,...,395.0,NaN,48.0,89.0,899,134,10,127,1066,693,570,21131,779,472,19880,139750,97600,NaN,42000,150,0,75,1278,1576,1847,159,75,5,11,66,70,68,NaN,2,1,0,1,9,3,3


In [ ]:
yield_df['Regions'] = yield_df.Regions.str.strip()

In [ ]:
yield_df.Regions.unique()

array(['NL01', 'LD01', 'LD02', 'LD03', 'LD04', 'PV20', 'PV21', 'PV22',
       'PV23', 'PV24', 'PV25', 'PV26', 'PV27', 'PV28', 'PV29', 'PV30',
       'PV31', 'CR01', 'CR02', 'CR03', 'CR04', 'CR05', 'CR06', 'CR07',
       'CR08', 'CR09', 'CR10', 'CR11', 'CR12', 'CR13', 'CR14', 'CR15',
       'CR16', 'CR17', 'CR18', 'CR19', 'CR20', 'CR21', 'CR22', 'CR23',
       'CR24', 'CR25', 'CR26', 'CR27', 'CR28', 'CR29', 'CR30', 'CR31',
       'CR32', 'CR33', 'CR34', 'CR35', 'CR36', 'CR37', 'CR38', 'CR39',
       'CR40', 'LG01', 'LG02', 'LG03', 'LG04', 'LG05', 'LG06', 'LG07',
       'LG08', 'LG09', 'LG10', 'LG11', 'LG12', 'LG13', 'LG14', 'LB2001',
       'LB2002', 'LB2003', 'LB2004', 'LB2005', 'LB2006', 'LB2101',
       'LB2102', 'LB2103', 'LB2104', 'LB2201', 'LB2202', 'LB2203',
       'LB2204', 'LB2205', 'LB2301', 'LB2302', 'LB2303', 'LB2304',
       'LB2401', 'LB2402', 'LB2501', 'LB2502', 'LB2503', 'LB2504',
       'LB2505', 'LB2506', 'LB2507', 'LB2508', 'LB2601', 'LB2602',
       'LB2603', 'LB2604

In [ ]:
province_map = {'PV20':	'Groningen',
'PV21':	'Fryslân',
'PV22':	'Drenthe',
'PV23':	'Overijssel',
'PV24':	'Flevoland',
'PV25':	'Gelderland',
'PV26':	'Utrecht',
'PV27':	'Noord-Holland',
'PV28':	'Zuid-Holland',
'PV29':	'Zeeland',
'PV30':	'Noord-Brabant',
'PV31':	'Limburg'}

In [ ]:
yield_province_df = yield_df.loc[yield_df['Regions'].isin(list(province_map.keys()))]

In [ ]:
yield_province_df.head()

,ID,FarmTypes,Regions,Periods,NumberOfFarmsTotal_1,LandUseTotal_2,UtilisedAgriculturalAreaUAA_3,OtherLand_4,LandUseTotal_5,UtilisedAgriculturalAreaUAA_6,OtherLand_7,ArableLandTotal_8,PotatoesTotal_9,WarePotatoes_10,SeedPotatoes_11,StarchPotatoes_12,VegetablesArable_13,Cereals_14,GrassSeed_15,IndustrialCrops_16,Pulses_17,SugarBeets_18,OtherArableCrops_19,FallowLand_20,ArableLandTotal_21,PotatoesTotal_22,WarePotatoes_23,SeedPotatoes_24,StarchPotatoes_25,VegetablesArable_26,Cereals_27,GrassSeed_28,IndustrialCrops_29,Pulses_30,SugarBeets_31,OtherArableCrops_32,FallowLand_33,HorticultureInTheOpenTotal_34,FlowerBulbs_35,FloriculturalCrops_36,...,Bulls2YearsTotal_111,BullsForBreeding2Years_112,BullsForFattening2Years_113,Sheep_114,GoatsTotal_115,GoatsForMilking_116,OtherGoats_117,EquidaeTotal_118,Horses_119,Ponies_120,PigsTotal_121,Piglets_122,PigsForBreeding_123,PigsForFattening_124,ChickensTotal_125,LayingHens_126,ParentBirdsOfLayingHens_127,Broilers_128,ParentBirdsOfBroilers_129,Turkeys_130,DucksForFattening_131,OtherPoultry_132,Rabbits_133,FurredAnimals_134,HousedAnimalsTotal_135,PigsTotal_136,Piglets_137,PigsForBreeding_138,PigsForFattening_139,ChickensTotal_140,LayingHens_141,ParentBirdsOfLayingHens_142,Broilers_143,ParentBirdsOfBroilers_144,Turkeys_145,DucksForFattening_146,OtherPoultry_147,Rabbits_148,FurredAnimals_149,Province
105,2982,A009481,PV20,2000JJ00,1449,8509601.0,7985637,523964.0,1449.0,1449,1278.0,7551365,2236745,192727,797322,1246696,122545,3383083,134106,124575,2358,1108125,79599,360229,1373,998,192,383,621,151,1283,140,100,9,1155,129,718,11614,1890,27,...,NaN,6.0,8.0,168,21,2,19,115,82,57,1533,1,115,1417,43200,43200,NaN,0,0,0,0,0,0,0,11,5,1,2,4,6,6,NaN,0,0,0,0,0,0,0,Groningen
106,2983,A009481,PV20,2001JJ00,1376,8242955.0,7772165,470790.0,1376.0,1376,1181.0,7304082,2108239,159397,761356,1187486,112345,3338960,115002,120574,4830,1052636,67802,383694,1295,921,159,352,582,135,1214,122,89,12,1103,99,741,10929,2539,10,...,NaN,4.0,10.0,173,21,0,21,115,79,55,2423,400,143,1880,56050,28050,NaN,28000,0,0,0,50,0,0,13,7,2,4,5,5,4,NaN,1,0,0,0,1,0,0,Groningen
107,2984,A009481,PV20,2002JJ00,1327,8145589.0,7676775,468814.0,1327.0,1327,1174.0,7169726,2064948,144362,751886,1168700,109425,3267153,90247,130054,6206,1041717,37535,422441,1241,874,134,310,536,130,1162,102,98,14,1044,59,746,7555,2268,20,...,NaN,2.0,11.0,155,25,2,23,105,72,53,1969,0,0,1969,71600,45600,NaN,26000,0,0,0,0,0,0,9,5,0,0,5,4,3,NaN,1,0,0,0,0,0,0,Groningen
108,2985,A009481,PV20,2003JJ00,1287,8172187.0,7688387,483800.0,1287.0,1287,1160.0,7271778,2117378,130603,768531,1218244,123466,3322260,102598,147721,4820,1008248,32032,413255,1220,841,119,299,510,133,1139,105,116,11,1031,61,725,10274,2175,20,...,NaN,3.0,12.0,148,21,2,20,106,74,53,2004,0,0,2004,32150,32150,NaN,0,0,0,0,158,0,0,7,4,0,0,4,2,2,NaN,0,0,0,0,1,0,0,Groningen
109,2986,A009481,PV20,2004JJ00,1272,8133977.0,7685531,448446.0,1272.0,1272,1128.0,7279393,2223103,120543,800485,1302075,144210,3356626,115736,120373,2034,980261,46247,290803,1213,819,109,287,504,146,1143,115,133,8,1003,58,746,8382,1618,0,...,NaN,4.0,6.0,146,18,3,17,115,71,67,1876,1,1,1874,36940,36940,NaN,0,0,0,0,0,0,0,10,5,1,1,4,5,5,NaN,0,0,0,0,0,0,0,Groningen


In [ ]:
yield_province_df["Province"] = yield_province_df["Regions"].map(province_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Get 1 weather station for each province

top_weather_station = gdf.groupby(['Nearest']).max()

In [ ]:
top_weather_station

,STN,LON,LAT,ALT,NAME
Nearest,,,,,
Drenthe,280,6.585,53.125,15.8,HOOGEVEEN
Flevoland,269,5.520,52.649,7.3,LELYSTAD
Fryslân,277,6.200,53.413,10.8,VLIELAND
Gelderland,356,6.657,52.069,48.2,HUPSEL
Groningen,286,7.150,53.575,0.0,NIEUW BEERTA
Limburg,391,6.197,51.498,114.3,MAASTRICHT
Noord-Brabant,375,5.707,51.659,22.6,WOENSDRECHT
Noord-Holland,257,5.174,52.928,8.5,IJMOND
Overijssel,290,6.891,52.703,34.8,TWENTHE


In [ ]:
province_yield_station_df = top_weather_station.merge(yield_province_df, how='left', left_on= 'Nearest', right_on= "Province")

In [ ]:
#Get year columns

province_yield_station_df['Year'] = province_yield_station_df['Periods'].astype(str).str[0:4]

In [ ]:
province_yield_station_df['Year']= pd.to_numeric(province_yield_station_df['Year'], errors='coerce')

In [ ]:
province_yield_station_df.head()

,STN,LON,LAT,ALT,NAME,ID,FarmTypes,Regions,Periods,NumberOfFarmsTotal_1,LandUseTotal_2,UtilisedAgriculturalAreaUAA_3,OtherLand_4,LandUseTotal_5,UtilisedAgriculturalAreaUAA_6,OtherLand_7,ArableLandTotal_8,PotatoesTotal_9,WarePotatoes_10,SeedPotatoes_11,StarchPotatoes_12,VegetablesArable_13,Cereals_14,GrassSeed_15,IndustrialCrops_16,Pulses_17,SugarBeets_18,OtherArableCrops_19,FallowLand_20,ArableLandTotal_21,PotatoesTotal_22,WarePotatoes_23,SeedPotatoes_24,StarchPotatoes_25,VegetablesArable_26,Cereals_27,GrassSeed_28,IndustrialCrops_29,Pulses_30,SugarBeets_31,...,BullsForBreeding2Years_112,BullsForFattening2Years_113,Sheep_114,GoatsTotal_115,GoatsForMilking_116,OtherGoats_117,EquidaeTotal_118,Horses_119,Ponies_120,PigsTotal_121,Piglets_122,PigsForBreeding_123,PigsForFattening_124,ChickensTotal_125,LayingHens_126,ParentBirdsOfLayingHens_127,Broilers_128,ParentBirdsOfBroilers_129,Turkeys_130,DucksForFattening_131,OtherPoultry_132,Rabbits_133,FurredAnimals_134,HousedAnimalsTotal_135,PigsTotal_136,Piglets_137,PigsForBreeding_138,PigsForFattening_139,ChickensTotal_140,LayingHens_141,ParentBirdsOfLayingHens_142,Broilers_143,ParentBirdsOfBroilers_144,Turkeys_145,DucksForFattening_146,OtherPoultry_147,Rabbits_148,FurredAnimals_149,Province,Year
0,280,6.585,53.125,15.8,HOOGEVEEN,3024,A009481,PV22,2000JJ00,1273,5767901.0,5422231,345670.0,1273.0,1273,914.0,5019269,2174222,159515,142206,1872501,40422,1532464,88177,9719,273,1032500,51008,90484,1157,938,164,152,892,43,1005,93,11,3,918,...,3.0,6.0,45,9,0,9,124,86,53,214,0,0,214,3750,3750,NaN,0,0,0,0,0,0,0,7,4,0,0,4,4,4,NaN,0,0,0,0,0,0,0,Drenthe,2000
1,280,6.585,53.125,15.8,HOOGEVEEN,3025,A009481,PV22,2001JJ00,1182,5588463.0,5280535,307928.0,1182.0,1182,856.0,4907536,2051005,121340,109438,1820227,32804,1583099,63363,16179,505,1011146,37970,111465,1073,862,130,124,824,37,939,71,14,2,874,...,5.0,3.0,39,15,2,14,125,80,61,880,0,0,880,3720,3720,NaN,0,0,0,0,0,0,0,9,6,0,0,6,3,3,NaN,0,0,0,0,0,0,0,Drenthe,2001
2,280,6.585,53.125,15.8,HOOGEVEEN,3026,A009481,PV22,2002JJ00,1134,5424696.0,5155152,269544.0,1134.0,1134,859.0,4694628,2004012,98329,87541,1818142,42129,1464204,50688,38197,186,960682,21765,112765,1009,806,111,71,766,43,875,53,44,2,809,...,1.0,4.0,33,7,0,7,120,82,65,514,11,6,497,0,0,NaN,0,0,0,0,0,0,0,5,5,1,1,4,0,0,NaN,0,0,0,0,0,0,0,Drenthe,2002
3,280,6.585,53.125,15.8,HOOGEVEEN,3027,A009481,PV22,2003JJ00,1081,5520069.0,5218703,301366.0,1081.0,1081,841.0,4735379,2036731,94545,78571,1863615,32764,1511506,48716,49430,1131,922029,21336,111736,973,750,93,57,718,38,856,53,55,4,767,...,2.0,3.0,32,8,0,8,105,69,57,300,0,0,300,90,90,NaN,0,0,0,0,0,0,0,4,2,0,0,2,2,2,NaN,0,0,0,0,0,0,0,Drenthe,2003
4,280,6.585,53.125,15.8,HOOGEVEEN,3028,A009481,PV22,2004JJ00,1045,5425304.0,5161268,264036.0,1045.0,1045,782.0,4728420,2159992,100355,83130,1976507,30591,1481084,72797,13631,1652,872356,15047,81270,942,732,102,55,696,32,830,65,25,5,744,...,0.0,1.0,28,10,0,10,99,70,48,430,0,0,430,0,0,NaN,0,0,0,0,0,0,0,4,4,0,0,4,0,0,NaN,0,0,0,0,0,0,0,Drenthe,2004


In [ ]:
# clean column names 

province_yield_station_df= province_yield_station_df.rename(columns=lambda x: x.strip())

# WEATHER DATA


# Column metadata
YYYYMMDD  = Datum (YYYY=jaar MM=maand DD=dag) / Date (YYYY=year MM=month DD=day)
DDVEC     = Vectorgemiddelde windrichting in graden (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil/variabel). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Vector mean wind direction in degrees (360=north, 90=east, 180=south, 270=west, 0=calm/variable)
FHVEC     = Vectorgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Vector mean windspeed (in 0.1 m/s)
FG        = Etmaalgemiddelde windsnelheid (in 0.1 m/s) / Daily mean windspeed (in 0.1 m/s)
FHX       = Hoogste uurgemiddelde windsnelheid (in 0.1 m/s) / Maximum hourly mean windspeed (in 0.1 m/s)
FHXH      = Uurvak waarin FHX is gemeten / Hourly division in which FHX was measured
FHN       = Laagste uurgemiddelde windsnelheid (in 0.1 m/s) / Minimum hourly mean windspeed (in 0.1 m/s)
FHNH      = Uurvak waarin FHN is gemeten / Hourly division in which FHN was measured
FXX       = Hoogste windstoot (in 0.1 m/s) / Maximum wind gust (in 0.1 m/s)
FXXH      = Uurvak waarin FXX is gemeten / Hourly division in which FXX was measured
TG        = Etmaalgemiddelde temperatuur (in 0.1 graden Celsius) / Daily mean temperature in (0.1 degrees Celsius)
TN        = Minimum temperatuur (in 0.1 graden Celsius) / Minimum temperature (in 0.1 degrees Celsius)
TNH       = Uurvak waarin TN is gemeten / Hourly division in which TN was measured
TX        = Maximum temperatuur (in 0.1 graden Celsius) / Maximum temperature (in 0.1 degrees Celsius)
TXH       = Uurvak waarin TX is gemeten / Hourly division in which TX was measured
T10N      = Minimum temperatuur op 10 cm hoogte (in 0.1 graden Celsius) / Minimum temperature at 10 cm above surface (in 0.1 degrees Celsius)
T10NH     = 6-uurs tijdvak waarin T10N is gemeten / 6-hourly division in which T10N was measured; 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT
SQ        = Zonneschijnduur (in 0.1 uur) berekend uit de globale straling (-1 voor <0.05 uur) / Sunshine duration (in 0.1 hour) calculated from global radiation (-1 for <0.05 hour)
SP        = Percentage van de langst mogelijke zonneschijnduur / Percentage of maximum potential sunshine duration
Q         = Globale straling (in J/cm2) / Global radiation (in J/cm2)
DR        = Duur van de neerslag (in 0.1 uur) / Precipitation duration (in 0.1 hour)
RH        = Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Daily precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
RHX       = Hoogste uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Maximum hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
RHXH      = Uurvak waarin RHX is gemeten / Hourly division in which RHX was measured
PG        = Etmaalgemiddelde luchtdruk herleid tot zeeniveau (in 0.1 hPa) berekend uit 24 uurwaarden / Daily mean sea level pressure (in 0.1 hPa) calculated from 24 hourly values
PX        = Hoogste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa) / Maximum hourly sea level pressure (in 0.1 hPa)
PXH       = Uurvak waarin PX is gemeten / Hourly division in which PX was measured
PN        = Laagste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa) / Minimum hourly sea level pressure (in 0.1 hPa)
PNH       = Uurvak waarin PN is gemeten / Hourly division in which PN was measured
VVN       = Minimum opgetreden zicht / Minimum visibility; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
VVNH      = Uurvak waarin VVN is gemeten / Hourly division in which VVN was measured
VVX       = Maximum opgetreden zicht / Maximum visibility; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
VVXH      = Uurvak waarin VVX is gemeten / Hourly division in which VVX was measured
NG        = Etmaalgemiddelde bewolking (bedekkingsgraad van de bovenlucht in achtsten, 9=bovenlucht onzichtbaar) / Mean daily cloud cover (in octants, 9=sky invisible)
UG        = Etmaalgemiddelde relatieve vochtigheid (in procenten) / Daily mean relative atmospheric humidity (in percents)
UX        = Maximale relatieve vochtigheid (in procenten) / Maximum relative atmospheric humidity (in percents)
UXH       = Uurvak waarin UX is gemeten / Hourly division in which UX was measured
UN        = Minimale relatieve vochtigheid (in procenten) / Minimum relative atmospheric humidity (in percents)
UNH       = Uurvak waarin UN is gemeten / Hourly division in which UN was measured
EV24      = Referentiegewasverdamping (Makkink) (in 0.1 mm) / Potential evapotranspiration (Makkink) (in 0.1 mm)


In [ ]:
# Average yearly temperatures
weather_data_df = pd.read_csv('/content/weather_KNMI_20181231.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,34,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
weather_data_df

,STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,FXXH,TG,TN,TNH,TX,TXH,T10N,T10NH,SQ,SP,Q,DR,RH,RHX,RHXH,PG,PX,PXH,PN,PNH,VVN,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24
0,209,20010130,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,209,20010131,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,209,20010201,236,13,24,50,8,10,1,90,20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,209,20010202,150,46,58,80,9,10,3,100,9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,209,20010203,106,65,66,90,8,50,1,120,8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804093,391,20181227,162,8,10,20,5,0,16,30,10,12,-18,8,47,14,-26,12,28,36,251,0,0,0,1,,,,,,,,,,,87,97,20,69,13,3
804094,391,20181228,229,17,18,30,21,10,1,50,21,7,-29,5,30,19,-44,12,23,29,206,0,0,0,1,,,,,,,,,,,91,98,3,83,17,2
804095,391,20181229,243,38,42,60,15,30,1,110,15,59,25,1,92,21,25,6,0,0,110,15,5,3,16,,,,,,,,,,,90,97,4,74,23,1
804096,391,20181230,255,24,26,40,1,20,4,100,1,78,52,9,87,14,38,12,3,4,148,42,17,6,17,,,,,,,,,,,89,99,21,78,1,2


In [ ]:
# Get year for the weather data 

weather_data_df['date'] = pd.to_datetime(weather_data_df['YYYYMMDD'],format='%Y%m%d', errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
weather_data_df['year'] = pd.DatetimeIndex(weather_data_df['date']).year

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# select weather only from year 2000

weather_data_df = weather_data_df[weather_data_df['year'] >= 2000]

In [ ]:
weather_data_df.head(2)

,STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,FXXH,TG,TN,TNH,TX,TXH,T10N,T10NH,SQ,SP,Q,DR,RH,RHX,RHXH,PG,PX,PXH,PN,PNH,VVN,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24,date,year
0,209,20010130,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2001-01-30,2001
1,209,20010131,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2001-01-31,2001


In [ ]:
weather_data_df.columns

Index(['STN', 'YYYYMMDD', 'DDVEC', 'FHVEC', '   FG', '  FHX', ' FHXH', '  FHN',
       ' FHNH', '  FXX', ' FXXH', '   TG', '   TN', '  TNH', '   TX', '  TXH',
       ' T10N', 'T10NH', '   SQ', '   SP', '    Q', '   DR', '   RH', '  RHX',
       ' RHXH', '   PG', '   PX', '  PXH', '   PN', '  PNH', '  VVN', ' VVNH',
       '  VVX', ' VVXH', '   NG', '   UG', '   UX', '  UXH', '   UN', '  UNH',
       ' EV24', 'date', 'year'],
      dtype='object')

In [ ]:
# remove white space 

weather_data_df= weather_data_df.rename(columns=lambda x: x.strip())

In [ ]:
weather_data_df = weather_data_df.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
weather_data_df['year'] = weather_data_df['year'].astype(int)

In [ ]:
# get average yearly temperature 
grouped_weather = weather_data_df.groupby(['STN','year'])['DDVEC', 'FHVEC', 'FG', 'FHX', 'FHXH', 'FHN', 'FHNH',
       'FXX', 'FXXH', 'TG', 'TN', 'TNH', 'TX', 'TXH', 'T10N', 'T10NH', 'SQ',
       'SP', 'Q', 'DR', 'RH', 'RHX', 'RHXH', 'PG', 'PX', 'PXH', 'PN', 'PNH',
       'VVN', 'VVNH', 'VVX', 'VVXH', 'NG', 'UG', 'UX', 'UXH', 'UN', 'UNH',
       'EV24'].mean().reset_index()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:

province_yield_station_df['STN']= province_yield_station_df['STN'].astype(str)
province_yield_station_df['Year']= province_yield_station_df['Year'].astype(str)

In [ ]:
grouped_weather['STN']= grouped_weather['STN'].astype(str)
grouped_weather['year']= grouped_weather['year'].astype(str)


In [ ]:
# join on year and station

merged_weather_province_yield = pd.merge(grouped_weather, province_yield_station_df,  how='inner', left_on=['STN','year'], right_on =['STN','Year'])

# GET ONLY CROP DATA (DROP ANIMAL DATA)

In [ ]:
print(merged_weather_province_yield.columns)

Index(['STN', 'year', 'DDVEC', 'FHVEC', 'FG', 'FHX', 'FHXH', 'FHN', 'FHNH',
       'FXX',
       ...
       'ParentBirdsOfLayingHens_142', 'Broilers_143',
       'ParentBirdsOfBroilers_144', 'Turkeys_145', 'DucksForFattening_146',
       'OtherPoultry_147', 'Rabbits_148', 'FurredAnimals_149', 'Province',
       'Year'],
      dtype='object', length=200)


In [ ]:
str_1 = 'CattleTotal_84	YoungAnimalsForMilk_85	CalvesForFattening_86	YoungAnimalsForFattening_87	DairyCows2Years_88	OtherCowsTotal_89	CowsForFattening2Years_90	SucklingCows2Years_91	Bulls2YearsTotal_92	BullsForBreeding2Years_93	BullsForFattening2Years_94	Sheep_95	GoatsTotal_96	GoatsForMilking_97	OtherGoats_98	EquidaeTotal_99	Horses_100	Ponies_101	GrazingLivestockTotal_102	CattleTotal_103	YoungAnimalsForMilk_104	CalvesForFattening_105	YoungAnimalsForFattening_106	DairyCows2Years_107	OtherCowsTotal_108	CowsForFattening2Years_109	SucklingCows2Years_110	Bulls2YearsTotal_111	BullsForBreeding2Years_112	BullsForFattening2Years_113	Sheep_114	GoatsTotal_115	GoatsForMilking_116	OtherGoats_117	EquidaeTotal_118	Horses_119	Ponies_120	PigsTotal_121	Piglets_122	PigsForBreeding_123	PigsForFattening_124	ChickensTotal_125	LayingHens_126	ParentBirdsOfLayingHens_127	Broilers_128	ParentBirdsOfBroilers_129	Turkeys_130	DucksForFattening_131	OtherPoultry_132	Rabbits_133	FurredAnimals_134	HousedAnimalsTotal_135	PigsTotal_136	Piglets_137	PigsForBreeding_138	PigsForFattening_139	ChickensTotal_140	LayingHens_141	ParentBirdsOfLayingHens_142	Broilers_143	ParentBirdsOfBroilers_144	Turkeys_145	DucksForFattening_146	OtherPoultry_147	Rabbits_148	FurredAnimals_149'
chunks = str_1.split('\t')


In [ ]:
merged_weather_province_yield.drop(chunks, axis=1, inplace= True)

In [ ]:
merged_weather_province_yield

,STN,year,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,FXXH,TG,TN,TNH,TX,TXH,T10N,T10NH,SQ,SP,Q,DR,RH,RHX,RHXH,PG,PX,PXH,PN,PNH,VVN,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,...,HorticultureUnderGlassTotal_46,FloriculturalCrops_47,NurseriesAndPerennialPlants_48,FruitUnderGlass_49,VegetablesUnderGlass_50,HorticultureUnderGlassTotal_51,FloriculturalCrops_52,NurseriesAndPerennialPlants_53,FruitUnderGlass_54,VegetablesUnderGlass_55,Hyacinths_56,Daffodils_57,Tulips_58,OtherBulbForcing_59,Mushrooms_60,SubstrateUseMushrooms_61,BelgianEndiveCultivation_62,BulbForcingTotal_63,Hyacinths_64,Daffodils_65,Tulips_66,OtherBulbForcing_67,MushroomsTotal_68,Mushrooms_69,SubstrateUseMushrooms_70,BelgianEndiveCultivation_71,GrasslandAndForagePlantsTotal_72,GrasslandTotal_73,PastureAndMeadows_74,RoughGrazing_75,TemporaryGrass_76,ForagePlants_77,GrasslandAndForagePlantsTotal_78,GrasslandTotal_79,PastureAndMeadows_80,RoughGrazing_81,TemporaryGrass_82,ForagePlants_83,Province,Year
0,257,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.342857,97.971429,10.726531,164.697959,12.910204,89.836735,13.567347,55.791837,37.902326,1243.471311,22.672727,28.848980,11.832653,7.910204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.289796,91.579592,10.330612,65.359184,12.559184,...,100,100,0,0,0,1,1,0,0,0,NaN,0,0,NaN,0,0,0,0,NaN,0,0,NaN,0,0,0,0,329427,222211,185706,8421,28084,107216,391,293,240,16,53,126,Noord-Holland,2001
1,257,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.342857,97.971429,10.726531,164.697959,12.910204,89.836735,13.567347,55.791837,37.902326,1243.471311,22.672727,28.848980,11.832653,7.910204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.289796,91.579592,10.330612,65.359184,12.559184,...,6900,0,0,3550,3350,8,0,0,4,5,NaN,0,0,NaN,0,0,0,0,NaN,0,0,NaN,0,0,0,0,2298,1923,388,0,1535,375,11,10,4,0,6,1,Noord-Holland,2001
2,257,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.547945,76.019178,11.276712,138.830137,13.010959,69.230137,13.347945,49.093151,38.501370,1033.084932,18.717808,23.849315,9.200000,7.071233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.638356,92.106849,9.978082,68.791781,12.926027,...,100,100,0,0,0,1,1,0,0,0,0.0,0,250,NaN,0,0,0,1,0.0,0,1,NaN,0,0,0,0,370534,264392,214289,16779,33324,106142,403,312,265,8,58,129,Noord-Holland,2002
3,257,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.547945,76.019178,11.276712,138.830137,13.010959,69.230137,13.347945,49.093151,38.501370,1033.084932,18.717808,23.849315,9.200000,7.071233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.638356,92.106849,9.978082,68.791781,12.926027,...,6386,0,0,2572,3814,6,0,0,3,5,0.0,0,0,NaN,0,0,0,0,0.0,0,0,NaN,0,0,0,0,2579,2204,448,531,1225,375,12,11,5,2,5,1,Noord-Holland,2002
4,257,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.682192,62.830137,11.835616,141.616438,13.164384,55.882192,13.627397,58.419178,45.515068,1146.805479,13.728767,18.753425,7.536986,5.865753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.876712,91.986301,10.493151,63.372603,13.117808,...,0,0,0,0,0,0,0,0,0,0,0.0,0,0,NaN,0,0,0,0,0.0,0,0,NaN,0,0,0,0,312012,211275,163709,12899,34667,100737,359,273,217,10,65,116,Noord-Holland,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,391,2016,186.661202,26.237705,29.639344,48.032787,10.696721,12.431694,9.008197,94.508197,12.303279,109.207650,66.062842,11.489071,153.800546,13.983607,53.513661,13.918033,49.270492,39.330601,1047.663934,17.592896,21.620219,8.636612,6.183060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.636612,95.781421,10.240437,61.303279,13.549180,...,103627,5000,2150,92577,3900,16,1,1,13,3,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,32060,21200,13763,2451,4986,10860,121,111,80,8,53,33,Limburg,2016
450,391,2017,198.624658,24.843836,28.443836,45.917808,10.293151,12.767123,8.479452,93.890411,12.128767,111.810959,67.038356,11.10

In [ ]:
merged_weather_province_yield['FarmTypes'].unique()

array(['A009481', 'A009497'], dtype=object)

In [ ]:
# GET FARM TYPE SPECIALIST FIELD CROPS = A009481

merged_weather_province_yield_special = merged_weather_province_yield[merged_weather_province_yield['FarmTypes']=='A009481']

In [ ]:
merged_weather_province_yield_special.to_csv('/content/merged_weather_yield_specialist_field_crop.csv')

In [ ]:
merged_weather_province_yield_perm = merged_weather_province_yield[merged_weather_province_yield['FarmTypes']=='A009497']
merged_weather_province_yield_perm.to_csv('/content/merged_weather_yield_specialist_pemanent_crop.csv')

In [ ]:
# There are two farm types, group by farm type
merged_weather_province_yield.sort_values(by=['FarmTypes'])


,STN,year,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,FXXH,TG,TN,TNH,TX,TXH,T10N,T10NH,SQ,SP,Q,DR,RH,RHX,RHXH,PG,PX,PXH,PN,PNH,VVN,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,...,HorticultureUnderGlassTotal_46,FloriculturalCrops_47,NurseriesAndPerennialPlants_48,FruitUnderGlass_49,VegetablesUnderGlass_50,HorticultureUnderGlassTotal_51,FloriculturalCrops_52,NurseriesAndPerennialPlants_53,FruitUnderGlass_54,VegetablesUnderGlass_55,Hyacinths_56,Daffodils_57,Tulips_58,OtherBulbForcing_59,Mushrooms_60,SubstrateUseMushrooms_61,BelgianEndiveCultivation_62,BulbForcingTotal_63,Hyacinths_64,Daffodils_65,Tulips_66,OtherBulbForcing_67,MushroomsTotal_68,Mushrooms_69,SubstrateUseMushrooms_70,BelgianEndiveCultivation_71,GrasslandAndForagePlantsTotal_72,GrasslandTotal_73,PastureAndMeadows_74,RoughGrazing_75,TemporaryGrass_76,ForagePlants_77,GrasslandAndForagePlantsTotal_78,GrasslandTotal_79,PastureAndMeadows_80,RoughGrazing_81,TemporaryGrass_82,ForagePlants_83,Province,Year
0,257,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.342857,97.971429,10.726531,164.697959,12.910204,89.836735,13.567347,55.791837,37.902326,1243.471311,22.672727,28.848980,11.832653,7.910204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.289796,91.579592,10.330612,65.359184,12.559184,...,100,100,0,0,0,1,1,0,0,0,NaN,0,0,NaN,0,0,0,0,NaN,0,0,NaN,0,0,0,0,329427,222211,185706,8421,28084,107216,391,293,240,16,53,126,Noord-Holland,2001
260,331,2017,206.989041,49.857534,58.287671,84.739726,11.205479,32.356164,10.789041,117.178082,11.586301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,445,0,0.0,0,0,0.0,0,0,0,2,1032866,698234,236858,147947,313429,334632,1180,1061,788,58,564,401,Zeeland,2017
258,331,2016,194.644258,49.705882,56.960784,81.652661,10.840336,31.680672,10.599440,114.061625,11.719888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1500,1500,0,0,0,1,1,0,0,0,0.0,0,0,0.0,0,0,400,0,0.0,0,0,0.0,0,0,0,1,976891,645893,243369,160078,242446,330998,1185,1055,792,65,536,400,Zeeland,2016
256,331,2015,197.452055,53.652055,61.860274,89.041096,11.334247,34.876712,10.646575,123.342466,12.235616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,300,0,0,1000,2,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,965642,607607,253307,105345,248955,358035,1258,1108,816,55,556,467,Zeeland,2015
254,331,2014,194.789041,51.136986,58.602740,83.808219,11.246575,33.232877,10.526027,116.356164,12.131507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,300,300,0,0,0,1,1,0,0,0,0.0,0,0,0.0,0,0,301,0,0.0,0,0,0.0,0,0,0,2,890416,585044,318930,154213,111901,305372,1157,1017,787,67,415,374,Zeeland,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,277,2007,206.328767,58.942466,67.386301,97.068493,11.364384,37.205479,10.350685,138.849315,11.945205,111.358904,86.739726,11.054795,136.200000,13.041096,80.054795,13.430137,47.082192,36.071233,1001.830137,18.753425,27.794521,11.441096,7.208219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.109589,92.194521,10.490411,71.797260,12.994521,...,0,0,0,0,0,0,0,0,0,0,0.0,0,0,NaN,0,0,0,0,0.0,0,0,NaN,0,0,0,0,408,408,408,0,0,0,2,2,2,0,0,0,Fryslân,2007
275,344,2005,203.432877,37.287671,42.232877,64.849315,11.290411,19.287671,9.208219,111.671233,11.561644,108.709589,68.032877,11.189041,146.249315,13.536986,51.728767,13.775342,49.501370,38.958904,1040.942466,15.764384,22.753425,8.449315,8.098630,10168.953425,10198.106849,12.830137,10138.528767,12.224658,35.041096,11.084932,73.216438,12.016438,5.126027,81.49

In [ ]:
# move province
merged_weather_province_yield.to_csv('/content/merged_weather_yield.csv')